## 2. Zapis zmiennych kinematycznych
W tym kroku używamy otrzymane wcześniej pliki do stworzenia zbiorów danych ze zmiennymi kinematycznymi dla każdego zdarzenia. Zbiór zapisujemy do formatu .csv

In [ ]:
import csv
import numpy as np

In [ ]:
def loading_data_restructured(file):
    with open(file, 'r', encoding='utf-8') as f:
        all_events = []
        reader = csv.reader(f)
        header = next(reader)  # pomijamy nagłówek

        for row in reader:
            event_num = int(row[0])           # numer zdarzenia
            interaction_type = row[1]         # typ interakcji (CC / NC)
            particles = []

            # dane cząstek (pid, px, py, pz, E)
            for i in range(3, len(row), 5):
                try:
                    pid = int(row[i])
                    px = float(row[i+1])
                    py = float(row[i+2])
                    pz = float(row[i+3])
                    E  = float(row[i+4])
                    M = np.sqrt(max(E**2 - (px**2 + py**2 + pz**2), 0))
                    particles.append([pid, px, py, pz, E, M])
                except (ValueError, IndexError):
                    continue

            if not particles:
                continue

            # całkowite wielkości dla zdarzenia
            total_px = sum(p[1] for p in particles)
            total_py = sum(p[2] for p in particles)
            total_pz = sum(p[3] for p in particles)
            total_E  = sum(p[4] for p in particles)
            total_M  = np.sqrt(max(total_E**2 - (total_px**2 + total_py**2 + total_pz**2), 0))

            event = [event_num, interaction_type, total_px, total_py, total_pz, total_E, total_M]
            for p in particles:
                event.extend(p)

            all_events.append(event)

    return all_events

Następnie filtrujemy zdarzenia, aby uzyskać tło oraz sygnał.
Chcemy wybrać tylko zdarzenia zawierające:
- dokładnie **1 proton (PID = 2212)**,  
- dokładnie **3 piony (PID = 211 lub -211)**,  
- razem 4 cząstki w zdarzeniu.  

Poniżej dwa warianty:
- `CC` (sygnał)  
- `NC` (tło)  

In [ ]:
def filter_proton_3pion_CC(events):
    filtered = []
    for event in events:
        if event[1] != "CC":
            continue
        pids = [int(event[i]) for i in range(7, len(event), 6)] # Czytamy kody PDG cząstek w zdarzeniu
        if pids.count(2212) == 1 and (pids.count(211) + pids.count(-211)) == 3 and len(pids) == 4: # Tylko zdarzenia 3pi+p
            filtered.append(event)
    return filtered


def filter_proton_3pion_NC(events):
    filtered = []
    for event in events:
        if event[1] != "NC":
            continue
        pids = [int(event[i]) for i in range(7, len(event), 6)]
        if pids.count(2212) == 1 and (pids.count(211) + pids.count(-211)) == 3 and len(pids) == 4:
            filtered.append(event)
    return filtered

Obliczamy sferyczność zdarzenia.

In [ ]:
def compute_sphericity(event):
    particles = []
    for i in range(7, len(event), 6):
        try:
            px = event[i+1]
            py = event[i+2]
            pz = event[i+3]
            particles.append([px, py, pz])
        except IndexError:
            continue

    if len(particles) < 2:
        return 0

    S = np.zeros((3, 3)) #Tworzymy macierz 3x3
    denominator = 0
    # Liczymy macierz sferyczności zgodnie ze wzorem
    for p in particles:
        p_vec = np.array(p)
        denominator += np.dot(p_vec, p_vec) 
        for i in range(3):
            for j in range(3):
                S[i][j] += p_vec[i] * p_vec[j]

    if denominator == 0:
        return 0

    S /= denominator
    # Liczymy wartości własne macierzy i sortujemy je od największej do najmniejszej
    eigenvalues = np.linalg.eigvalsh(S)
    eigenvalues = sorted(eigenvalues, reverse=True)
    # Liczymy sferyczność
    sphericity = 1.5 * (eigenvalues[1] + eigenvalues[2])
    return sphericity

Następnie zapisujemy parametry zdarzenia do pliku. 
Dla każdego zdarzenia zapisujemy:
- typ interakcji (CC / NC),  
- energię całkowitą `E`,  
- sferyczność,  
- poprzeczny pęd `pT`,  
- kąt względem osi z,  
- składową `Pz`.  
Typ interakcji zapisuje w celach kontrolnych, w dalszej analizie jest ignorowany.

In [ ]:
def save_events_to_csv(events, output_filename, max_events=50000):
    with open(output_filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Typ', 'E', 'Sferyczność', 'pT', 'Kąt z (deg)', 'Pz'])

        for event in events[:max_events]:
            typ = event[1]
            E = event[5]
            Px = event[2]
            Py = event[3]
            Pz = event[4]

            pT = np.sqrt(Px**2 + Py**2)
            p = np.array([Px, Py, Pz])
            angle_z_deg = np.degrees(np.arccos(Pz / (np.linalg.norm(p) + 1e-8)))

            spher = compute_sphericity(event)

            writer.writerow([typ, E, spher, pT, angle_z_deg, Pz])

Ręcznie ustawiam liczbę zdarzeń zapisywanych do plku "signal.csv" oraz "bgd.csv". Liczba zdarzeń jest obliczona w taki sposób, aby po odjęciu od obu zbiorów 50 000 zdarzeń, czyli liczby zdarzeń używanych w treningu, w zbiorach pozostawała liczba zdarzeń odpowiadająca stosunkowi sygnału do tła.

In [ ]:
# Pliki wejściowe
file_CC = "nuTau_data_CCQES.csv"
file_NC = "nuTau_data_NC_6mil.csv"

# Liczba zdarzeń do zapisania
N_signal = 59_685
N_bgd = 126_092

# Wczytywanie
all_events_CC = loading_data_restructured(file_CC)
all_events_NC = loading_data_restructured(file_NC)

# Filtrowanie
signal_events = filter_proton_3pion_CC(all_events_CC)
bgd_events = filter_proton_3pion_NC(all_events_NC)

# Zapis wyników
save_events_to_csv(signal_events, "signal.csv", max_events=N_signal)
save_events_to_csv(bgd_events, "bgd.csv", max_events=N_bgd)

print(f"Zapisano {min(len(signal_events), N_signal)} zdarzeń do 'signal.csv'")
print(f"Zapisano {min(len(bgd_events), N_bgd)} zdarzeń do 'bgd.csv'")